# Bifrost Neural Engine
A generic autonomous model builder for various problems using neural networks.

Problem Areas:
- Regression
- Classification
- Time Series Predictions

## Import Dependencies

In [7]:
%run ./../../utilities/common/data_loader.ipynb

In [16]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import time
import logging as log
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

## Implementation

In [9]:
class BifrostNeuralEngine(nn.Module):
    # TODO: Override load and save to persist and load data loader state dictionary.

    def __init__(self, 
                 data: pd.DataFrame, 
                 labels_column_names: list,
                 date_column_name: str = None,
                 layers_config: list = list(), 
                 dropout_probability: float=0.5, 
                 learning_rate: float=0.001):
        super().__init__()
        
        self.data_loader = DataLoader(data=data,
                                      labels_column_names=labels_column_names,
                                      date_column_name=date_column_name)
        train_x, train_y, _, _ = self.data_loader.get_train_test_split()
        self.features_column_names = train_x.columns
        self.output_count = train_y.shape[1]
        self.learning_rate = learning_rate
        self.labels_column_names = labels_column_names
        
        # Batch normalizer.
        self.bn_cont = nn.BatchNorm1d(len(self.features_column_names))
        
        # Layers.
        layers = list()
        n_in = len(self.features_column_names)
        
        if len(layers_config) <= 0:
            layers_config = self.__auto_determine_layers_config__()
        
        for i in layers_config:
            layers.append(nn.Linear(in_features=n_in, out_features=i))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.BatchNorm1d(num_features=i))
            layers.append(nn.Dropout(p=dropout_probability))
            n_in = i
            
        layers.append(nn.Linear(in_features=layers_config[-1], out_features=self.output_count))
        
        self.layers = nn.Sequential(*layers)

    def __auto_determine_layers_config__(self) -> list:
        raise NotImplementedError('TODO: Intelligent layer config generation.')

    def __get_tensors_for_dataframe__(self, data: pd.DataFrame) -> torch.tensor:
        return torch.tensor(data.values, dtype=torch.float)

    def __eval_test_data__(self, test_x: torch.tensor, test_y: torch.tensor, criterion: nn.modules.loss._Loss):
        log.info(f'Evaluating test data.')

        with torch.no_grad():
            y_pred = self(test_x)
            loss = torch.sqrt(criterion(y_pred, test_y))

        log.info(f'Training loss evaluation: {loss}')

    def forward(self, x: torch.tensor):
        x = self.bn_cont(x)
        x = self.layers(x)
        
        return x

    def fit(self, epochs: int):
        start_time: int = time.time()
        criterion: nn.modules.loss._Loss = nn.MSELoss()# if self.output_count == 1 else nn.CrossEntropyLoss()
        optimizer: torch.optim.Optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        train_x, train_y, test_x, test_y = self.data_loader.get_train_test_split()
        train_x = self.__get_tensors_for_dataframe__(data=train_x)
        train_y = self.__get_tensors_for_dataframe__(data=train_y)
        test_x = self.__get_tensors_for_dataframe__(data=test_x)
        test_y = self.__get_tensors_for_dataframe__(data=test_y)
        losses = list()

        for i in tqdm(range(epochs)):
            i += 1
            y_pred = self(train_x)
            loss = torch.sqrt(criterion(y_pred, train_y))
            losses.append(loss)

            if i % 100 == 1:
                log.debug(f'Epoch: {i}, Loss: {loss}')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        duration = time.time() - start_time
        log.info(f'Training took: {duration / 60} minutes.')

        self.__eval_test_data__(test_x=test_x, test_y=test_y, criterion=criterion)
        plt.plot(range(epochs), [float(l) for l in losses])

        return self

    def predict(self, x: pd.DataFrame) -> pd.Series:
        x = self.data_loader.get_featurized_data(data=x)
        x = x[[c for c in x.columns if c in self.features_column_names]]
        x = self.__get_tensors_for_dataframe__(data=x)

        with torch.no_grad():
            predictions = self.forward(x)
            predictions = pd.Series([pred.item() for pred in predictions])
            
            return predictions